In [ ]:
import pandas as pd
import requests

## US state population 2022

In [ ]:
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States_by_population")

In [ ]:
tables[0].columns[5]

In [ ]:
population = tables[0][[("State or territory", "State or territory"), ('Census population[8][a]', 'July 1, 2022 (est.)'), ('Census population[8][a]', 'April 1, 2010')]][:56].reset_index(drop=True)
population.columns = ["Territory", "Population", "Population 2010"]
population["Territory"] = population["Territory"].str.replace(pat="\[.*\]", repl="", regex=True)

# Courtesy of https://gist.github.com/rogerallen/1583593
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
    
population = population.assign(Code=population["Territory"].map(us_state_to_abbrev))

# invert the dictionary
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))


population.to_csv("us_state_population.csv", index=False)

## U.S. Cities

In [ ]:
response = requests.get("https://github.com/kelvins/US-Cities-Database/raw/main/csv/us_cities.csv")
with open("us-cities.csv", "w") as out:
    out.write(response.text)

In [ ]:
response = requests.get("https://github.com/plotly/datasets/raw/master/us-cities-top-1k.csv")
with open("us-cities-with-population.csv", "w") as out:
    out.write(response.text)

## Laser incidents in 2019

From https://www.faa.gov/about/initiatives/lasers/laws/.

In [ ]:
response = requests.get("https://www.faa.gov/about/initiatives/lasers/laws/media/Laser_Report_2019_final.xlsx")
with open("laser_incidents_2019.xlsx", "wb") as xls_out:
    xls_out.write(response.content)

In [ ]:
incidents = pd.read_excel("laser_incidents_2019.xlsx")

In [20]:
laser_incidents_raw = pd.read_csv("./laser_incidents_2015-2020_raw.csv")

C:\Users\janpi\AppData\Local\Temp\ipykernel_26124\1691797962.py:1: DtypeWarning: Columns (2,10,68) have mixed types. Specify dtype option on import or set low_memory=False.
  laser_incidents_raw = pd.read_csv("./laser_incidents_2015-2020_raw.csv")


In [21]:
laser_incidents = laser_incidents_raw[
    [column for column in laser_incidents_raw.columns if "Unnamed" not in column]
]
laser_incidents.columns

Index(['Incident Date', 'Incident Time', 'Flight ID', 'Aircraft', 'Altitude',
       'Airport', 'Laser Color', 'Injury', 'City', 'State',
       'Aviation Altitude', 'State '],
      dtype='object')

In [22]:
# there are "State" and "State " columns: merge them into a single one
laser_incidents = laser_incidents.assign(
    State=laser_incidents["State"].where(
        laser_incidents["State"].notna(), laser_incidents["State "]
    )
)

In [23]:
# strip white space from state names
laser_incidents = laser_incidents.assign(State=laser_incidents["State"].str.strip())

In [24]:
# drop columns we do not need any more ("Aviation Altitude" are all NA values)
laser_incidents = laser_incidents.drop(columns=["State ", "Aviation Altitude"])

In [25]:
import collections

# needed below for mapping string values to bool
# "yes" and "no" are defined, anything else becomes NA
value_to_bool = collections.defaultdict(lambda: pd.NA)
value_to_bool["yes"] = True
value_to_bool["no"] = False


# try to convert to better dtypes
laser_incidents = laser_incidents.convert_dtypes()
# convert some columns manually with some preprocessing
laser_incidents = laser_incidents.assign(
    **{
        "Incident Time": laser_incidents["Incident Time"].astype("string"),
        "Altitude": pd.to_numeric(laser_incidents["Altitude"], errors="coerce"),
        "Laser Color": laser_incidents["Laser Color"].str.strip().str.lower(),
        "Injury": laser_incidents["Injury"]
            .str.lower()
            .str.strip()
            .map(value_to_bool)
            .astype("boolean"),
    }
)

In [30]:
laser_incidents.loc[
    laser_incidents["Incident Time"].astype("string").str.len() > 4, "Incident Time"
]
# make the suspicious times NA
laser_incidents.loc[
    laser_incidents["Incident Time"].astype("string").str.len() > 4, "Incident Time"
] = pd.NA
# using string manipulation and time deltas to construct full time stamps (date + time)
laser_incidents = laser_incidents.assign(
    timestamp = pd.to_datetime(laser_incidents["Incident Date"])
    + pd.to_timedelta(
        laser_incidents["Incident Time"].str[:-2]
        + "h"
        + laser_incidents["Incident Time"].str[-2:]
        + "min",
        errors="coerce",
    )
)

In [31]:
laser_incidents.to_csv("laser_incidents_2015-2020.csv")